# Ranking et search engine

C'est un petit exemple de ranking avec un très petit jeu de données, trop petit pour que le modèle soit performant, mais le code peut être réutilisé pour des exemples de taille raisonnable. C'est à dire probablement pas pour apprnedre un moteur de recherche.

In [1]:
%matplotlib inline

In [2]:
from papierstat.datasets import load_search_engine_dataset
X, y, qid = load_search_engine_dataset()
X[:5,:6].todense()

matrix([[3., 3., 0., 0., 3., 1.],
        [3., 0., 3., 0., 3., 1.],
        [3., 0., 2., 0., 3., 1.],
        [3., 0., 3., 0., 3., 1.],
        [3., 0., 3., 0., 3., 1.]])

In [3]:
X.shape

(582, 136)

Le tableau ``qid`` contient l'identifiant de la requête, toutes les lignes associées à un identifiant correspondent à des résultats associés à cette requête. Dans ce jeu, il y a 7 requêtes distinctes.

In [4]:
set(qid)

{1, 16, 31, 46, 61, 76, 91}

On peut essayer d'abord [XGBoost](http://xgboost.readthedocs.io/en/latest/python/index.html). Ce petit jeu de données est aussi disponible sur [github/papierstat/datasets/data](https://github.com/sdpython/papierstat/tree/master/src/papierstat/datasets/data).

In [5]:
X_train, y_train, qid_train = load_search_engine_dataset()

In [6]:
import pandas
df = pandas.DataFrame(qid_train)
df['c'] = 1
df.columns = ['qid', 'c']
gr = df.groupby('qid').count()
gr

,c
qid,
1,86
16,106
31,92
46,120
61,59
76,45
91,74


In [7]:
from xgboost import DMatrix
dtrain = DMatrix(data=X_train, label=y_train)
dtrain.set_group(gr.as_matrix())

In [8]:
from xgboost import XGBRegressor, train
rk = train(params={'objective': 'rank:ndcg'}, dtrain=dtrain, num_boost_round=10)

In [9]:
X_test, y_test, qid_test = load_search_engine_dataset(False)

In [10]:
import pandas
df = pandas.DataFrame(qid_test)
df['c'] = 1
df.columns = ['qid', 'c']
gr = df.groupby('qid').count()
dtest = DMatrix(data=X_test)
dtest.set_group(gr.as_matrix())

In [11]:
pred = rk.predict(dtest)

On peut calculer l'erreur au carré.

In [12]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, pred)

0.9693970339752199

Mais cela n'est valeur que si le score a un sens, ce qui est le cas ici. Si ce n'est pas le cas, il est possible d'évaluer les résultats avec la corrélation des rangs des résultats ([coefficient de Kendall](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient)). Le module [pyltr](https://github.com/jma127/pyltr) est une autre option.

In [13]:
from pyltr.models import LambdaMART
from pyltr.metrics import NDCG
from pyltr.models.monitors import ValidationMonitor

metric = NDCG(k=5)

model = LambdaMART(metric=metric, n_estimators=10, learning_rate=0.02,
                   max_features=0.5, query_subsample=0.5, max_leaf_nodes=10,
                   min_samples_leaf=64, verbose=1)

monitor = ValidationMonitor(X_train.toarray(), y_train, qid_train,
                            metric=metric, stop_after=250)

model.fit(X_train.toarray(), y_train, qid_train, monitor=monitor)

 Iter  Train score  OOB Improve    Remaining                           Monitor Output 
    1       0.0131       0.0109        0.15s      C:      0.0119 B:      0.0119 S:  0
    2       0.2199       0.0855        0.14s      C:      0.1494 B:      0.1494 S:  0
    3       0.5021       0.0491        0.11s      C:      0.2857 B:      0.2857 S:  0
    4       0.2859       0.0081        0.09s      C:      0.2941 B:      0.2941 S:  0
    5       0.2955       0.0151        0.07s      C:      0.3038 B:      0.3038 S:  0
    6       0.1471       0.0445        0.06s      C:      0.3480 B:      0.3480 S:  0
    7       0.3106       0.0140        0.04s      C:      0.3787 B:      0.3787 S:  0
    8       0.5424       0.0469        0.03s      C:      0.4892 B:      0.4892 S:  0
    9       0.5948      -0.0032        0.01s      C:      0.4790 B:      0.4892 S:  1
Early termination at iteration  9


In [14]:
Epred = model.predict(X_test.toarray())

In [15]:
print('Random ranking:', metric.calc_mean_random(qid_test, y_test))
print('Our model:', metric.calc_mean(qid_test, y_test, Epred))

Random ranking: 0.22412952989302481
Our model: 0.17290185413803674


Comme prévu, ça ne marche pas sur un si petit jeu. Autre option [lightfm](https://github.com/lyst/lightfm) (article : [Learning to Rank Sketchfab Models with LightFM](http://blog.ethanrosenthal.com/2016/11/07/implicit-mf-part-2/)). [scikit-learn](http://scikit-learn.org/stable/) ne propose pas de modèle de ranking, il faut implémenter soi-même la transformation des données.